In [ ]:
# default_exp models.phy

# PhyDNet
> ConvLSTM + PhyCell
https://github.com/vincent-leguen/PhyDNet/blob/master/models/models.py

In [ ]:
#export
from fastai.vision.all import *

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(1)
    print(torch.cuda.get_device_name())

GeForce RTX 2070 SUPER


## The Phycell

In [ ]:
class PhyCell_Cell(Module):
    def __init__(self, ch_in, hidden_dim, ks=3, bias=True):
        store_attr()
        padding = ks // 2
        bias = bias
        self.f = nn.Sequential(
                 nn.GroupNorm(4, ch_in),   
                 nn.Conv2d(ch_in, hidden_dim, ks, padding=padding),
                 nn.Conv2d(hidden_dim, ch_in, kernel_size=(1,1)))

        self.convgate = nn.Conv2d(2*ch_in,
                                  ch_in,
                                  kernel_size=(3,3),
                                  padding=(1,1), 
                                  bias=bias)

    def forward(self, x, hidden): 
        "x ~[batch_size, hidden_dim, height, width]"  
        hidden_tilde = hidden + self.f(hidden)         # prediction
        combined = torch.cat([x, hidden_tilde], dim=1) # concatenate along channel axis
        combined_conv = self.convgate(combined)
        K = torch.sigmoid(combined_conv)
        next_hidden = hidden_tilde + K * (x-hidden_tilde)   # correction , Haddamard product     
        return next_hidden

In [ ]:
p_cell = PhyCell_Cell(16, 32, 3)
p_cell

PhyCell_Cell(
  (f): Sequential(
    (0): GroupNorm(4, 16, eps=1e-05, affine=True)
    (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  )
  (convgate): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [ ]:
p_cell(torch.rand(1,16,10,10), torch.rand(1,16,10,10)).shape

torch.Size([1, 16, 10, 10])

In [ ]:
class PhyCell(Module):
    def __init__(self, ch_in, hidden_dims, ks, n_layers):
        store_attr() 
        self.H = None
        self.cell_list = nn.ModuleList()
        for i in range(0, self.n_layers):
            self.cell_list.append(PhyCell_Cell(ch_in=ch_in,
                                               hidden_dim=hidden_dims[i],
                                               ks=ks))                                     
       
    def forward(self, x, first_timestep=False): # input_ [batch_size, 1, channels, width, height]    
        bs, _, ch, w, h = x.data.size()
        assert ch == self.ch_in, "Input tensor has different channels dim than Cell"
        if (first_timestep):   
            self.initHidden(bs, w, h) # init Hidden at each forward start
            print('Create hidden: ', self.H[0].shape)
        for j, cell in enumerate(self.cell_list):
            if j==0: # bottom layer
                self.H[j] = cell(x, self.H[j])
            else:
                self.H[j] = cell(self.H[j-1],self.H[j])
        
        return self.H , self.H 
    
    def initHidden(self, bs, w, h):
        self.H = [] 
        for i in range(self.n_layers):
            self.H.append(one_param(self).new_zeros(bs, self.ch_in, w, h))

    def setHidden(self, H):
        self.H = H

In [ ]:
phy = PhyCell(16, [32], 3, 1)

In [ ]:
phy(torch.rand(8,1,16,12,12), True)

Create hidden:  torch.Size([8, 16, 12, 12])


RuntimeError: Tensors must have same number of dimensions: got 5 and 4

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_data.ipynb.
Converted 01_models.conv_rnn.ipynb.
Converted 02_models.dcn.ipynb.
Converted 02_models.transformer.ipynb.
Converted 02_tcn.ipynb.
Converted index.ipynb.
